In [1]:
const Grammar = Vector{Tuple{String, String}}

function generate_word(grammar::Grammar, start_symbol::String="S", max_depth::Int=15)
    if max_depth <= 0 return "" end
    rules = [rhs for (lhs, rhs) in grammar if lhs == start_symbol]
    if isempty(rules) return start_symbol end

    chosen_rhs = rand(rules)
    word = ""
    for char in chosen_rhs
        token = string(char)
        if isuppercase(char)
            sub = generate_word(grammar, token, max_depth - 1)
            if sub == "" && max_depth < 5 return "" end
            word *= sub
        else
            word *= token
        end
    end
    return word
end

global var_counter = 0
function get_new_var() global var_counter += 1; return "_X$var_counter" end

function to_cnf(input_grammar::Grammar)
    global var_counter = 0
    step1 = Grammar()
    term_map = Dict{Char, String}()
    function get_term(c)
        if haskey(term_map, c) return term_map[c] end
        v = get_new_var(); push!(step1, (v, string(c))); term_map[c] = v; return v
    end

    for (lhs, rhs) in input_grammar
        if length(rhs) == 1 && islowercase(rhs[1]) push!(step1, (lhs, rhs)); continue end
        toks = []
        for c in rhs
            push!(toks, islowercase(c) ? get_term(c) : string(c))
        end
        if length(toks) == 1 push!(step1, (lhs, toks[1]))
        else
            curr = lhs
            while length(toks) > 2
                f = popfirst!(toks); nv = get_new_var()
                push!(step1, (curr, f * nv)); curr = nv
            end
            push!(step1, (curr, toks[1] * toks[2]))
        end
    end

    unit_pairs = Set{Tuple{String, String}}()
    non_unit = Grammar()
    for (l, r) in step1
        is_u = (length(r)==1 && !islowercase(r[1])) || (startswith(r, "_") && !occursin(r"(_X\d+).+", r) && length(r) < 5)
        if is_u push!(unit_pairs, (l, r)) else push!(non_unit, (l, r)) end
    end

    changed = true
    while changed
        changed = false
        curr = collect(unit_pairs)
        for (a, b) in curr, (c, d) in curr
            if b == c && !((a, d) in unit_pairs) && a != d
                push!(unit_pairs, (a, d)); changed = true
            end
        end
    end

    final = copy(non_unit)
    for (p, child) in unit_pairs
        for (l, r) in non_unit
            if l == child push!(final, (p, r)) end
        end
    end
    return unique(final)
end

function cyk_parse(w::String, grammar::Grammar, start_symbol::String="S")
    n = length(w)
    if n == 0 return false end
    inv = Dict{String, Set{String}}()
    for (l, r) in grammar
        if !haskey(inv, r) inv[r] = Set{String}() end
        push!(inv[r], l)
    end
    T = [Set{String}() for _ in 1:n, _ in 1:n]
    for i in 1:n
        s = string(w[i])
        if haskey(inv, s) union!(T[i, 1], inv[s]) end
    end
    for l in 2:n, i in 1:(n-l+1), k in 1:(l-1)
        left, right = T[i, k], T[i+k, l-k]
        if isempty(left) || isempty(right) continue end
        for B in left, C in right
            rhs = B * C
            if haskey(inv, rhs) union!(T[i, l], inv[rhs]) end
        end
    end
    return start_symbol in T[1, n]
end

g1 = [
    ("Z", "A"), ("A", "ab"), ("A", "aBa"), ("A", "CDD"),
    ("B", "ab"), ("B", "aBa"), ("B", "DDD"),
    ("C", "ba"), ("C", "bDb"), ("C", "AB"),
    ("D", "ba"), ("D", "bDb"), ("D", "BB")
]

g2 = [
    ("Z", "A"), ("A", "ab"), ("A", "aBa"), ("A", "EHH"),
    ("B", "ab"), ("B", "aBa"), ("B", "FHH"),
    ("C", "ab"), ("C", "aDa"), ("C", "GHH"),
    ("D", "ab"), ("D", "aDa"), ("D", "HHH"),
    ("E", "ba"), ("E", "bGb"), ("E", "AD"),
    ("F", "ba"), ("F", "bHb"), ("F", "BD"),
    ("G", "ba"), ("G", "bGb"), ("G", "CD"),
    ("H", "ba"), ("H", "bHb"), ("H", "DD")
]

g3 = [
    ("S", "TTT"), ("S", "aSa"), ("S", "ab"),
    ("T", "SS"), ("T", "bTb"), ("T", "ba")
]

function compare_grammars(count=5)
    pairs = [
        ("G1", g1, "Z", "G2", g2, "Z"),
        ("G1", g1, "Z", "G3", g3, "S"),
        ("G2", g2, "Z", "G3", g3, "S")
    ]

    for (name1, g1_raw, s1, name2, g2_raw, s2) in pairs
        println("Сравнение: $name1 vs $name2")

        cnf1 = to_cnf(g1_raw)
        cnf2 = to_cnf(g2_raw)

        function run_direction(src_name, src_g, src_s, target_name)
            println("Источник: $src_name")

            match_count = 0
            valid_gen = 0

            for i in 1:count
                w = ""
                attempts = 0
                while w == "" && attempts < 20
                    w = generate_word(src_g, src_s, 14)
                    attempts += 1
                end
                if w == "" continue end
                valid_gen += 1

                res1 = cyk_parse(w, cnf1, s1)
                res2 = cyk_parse(w, cnf2, s2)
                match = (res1 == res2)

                println(rpad(w, 35), " | ", res1, " | ", res2, " | ", match)

                if res1 == res2 match_count += 1 end
            end
            if valid_gen > 0
                println("Совпадений: $match_count из $valid_gen")
            end
        end

        run_direction(name1, g1_raw, s1, name2)
        run_direction(name2, g2_raw, s2, name1)
    end
end

compare_grammars(50)

Сравнение: G1 vs G2
Источник: G1
aaba                                | true | true | true
aaba                                | true | true | true
aabbbabbbaaaabaaaababbabbabbabaaabaaaa | true | true | true
aaba                                | true | true | true
bbabbabaaabbbabbbababbaaaabb        | false | false | true
bbabbabaaababbbabbbbabaababbabaaabaaabbabbabaaaabaaaaaaabababbabbbbaabaaaabaabaababababbababbabbbabb | true | true | true
ab                                  | true | true | true
ab                                  | true | true | true
ab                                  | true | true | true
ab                                  | true | true | true
ab                                  | true | true | true
baabaababbbbabbbabababbbbbbabbbaabaaabaaabaabaabbbabaaabaabababaaabaaaaaaabaaaabaabaabaaaaaaaabaabbaaababbbbabbbbaaabaaabbabaabaabaabbbbbbab | false | false | true
ababaabaaabaabbabbabbababaababaabbbbbabbabaaabaaaaba | true | true | true
ababbabaababbabbabbabababaabbabb